In [1]:
# manipulation des données
import numpy as np
import pandas as pd

# matplotlib et seaborn pour les représentations graphiques
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# sklearn preprocessing pour le traiter les variables catégorielles
from sklearn.preprocessing import LabelEncoder

# Gestion du système de fichiers
import os

# Suppression des alertes
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../Data/2.sample_dataset.csv")
df = df.dropna(subset=['words'])

In [3]:
import tensorflow as tf
# import tensorflow_hub as hub
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

os.environ["TF_KERAS"]='1'
os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "UNCOMPRESSED"

2024-01-25 16:52:04.581137: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import shutil

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

2024-01-25 16:52:13.063419: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


In [16]:
def feature_USE_fct(sentences, b_size) :
    batch_size = b_size

    features = None

    for step in range(len(sentences)//batch_size) :
        idx = step*batch_size
        feat = embed(sentences[idx:idx+batch_size])

        if step ==0 :
            features = feat
        else :
            features = np.concatenate((features,feat))
    return features

In [27]:
batch_size = 10
sentences = df['words'].sample(100000, random_state=42)
sentences = sentences.to_list()

In [28]:
x_sentences = feature_USE_fct(sentences, batch_size)

In [29]:
from sklearn.model_selection import train_test_split

labels = df['target'].sample(100000, random_state=42)
labels = labels.values

X_train, X_test, y_train, y_test = train_test_split(x_sentences, labels, test_size=0.2, random_state=42)

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from sklearn.metrics import accuracy_score

In [30]:
embedding_dim = 24  # Taille de l'espace d'embedding
filter_sizes = [2, 3, 4]  # Tailles des filtres convolutifs
num_filters = 64  # Nombre de filtres par taille
dropout_rate = 0.5

In [32]:
model = Sequential()
model.add(Embedding(input_dim=len(x_sentences[0]), output_dim=embedding_dim, input_length=len(x_sentences[0])))

for filter_size in filter_sizes:
    model.add(Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu'))

model.add(GlobalMaxPooling1D())
model.add(tf.keras.layers.Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))

In [33]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
2250/2250 [==============================] - 117s 51ms/step - loss: 0.6932 - accuracy: 0.5005 - val_loss: 0.6931 - val_accuracy: 0.5027
Epoch 2/5
2250/2250 [==============================] - 108s 48ms/step - loss: 0.6932 - accuracy: 0.5015 - val_loss: 0.6931 - val_accuracy: 0.5027
Epoch 3/5
2250/2250 [==============================] - 108s 48ms/step - loss: 0.6932 - accuracy: 0.5021 - val_loss: 0.6931 - val_accuracy: 0.5027
Epoch 4/5
2250/2250 [==============================] - 109s 48ms/step - loss: 0.6932 - accuracy: 0.5019 - val_loss: 0.6931 - val_accuracy: 0.5027
Epoch 5/5
2250/2250 [==============================] - 109s 48ms/step - loss: 0.6932 - accuracy: 0.5022 - val_loss: 0.6931 - val_accuracy: 0.5027


In [35]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

In [37]:
y_pred = clf.predict(X_test)

In [38]:
accuracy = accuracy_score(y_test, y_pred)

print(accuracy)

0.75115
